# References
This code is referenced from Kunal Deshmukh's GitHub Repository:
https://github.com/kunalrdeshmukh/End-to-end-compression

It was a big help as it helped speed-up implementation of CNN End-to-End Compression Framework:
https://arxiv.org/abs/1708.00838


Image Compression framework :

[End to end image compression framework ](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7999241)

In [ ]:
# http://pytorch.org/
# !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
!pip install wheel setuptools

from os.path import exists
# from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
# platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
print(accelerator)

import torch

import os
import datetime
import torch
import torchvision
from torch import nn , optim
from torchvision import datasets, transforms
from torch.autograd import Variable

!pip install Pillow==8.2.0
!pip install PIL
!pip install image
!pip install lmdb
from PIL import Image

cu110
ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [ ]:
import math
irange = range


def make_grid(tensor, nrow=8, padding=2,
              normalize=False, range=None, scale_each=False, pad_value=0):
    """Make a grid of images.
    Args:
        tensor (Tensor or list): 4D mini-batch Tensor of shape (B x C x H x W)
            or a list of images all of the same size.
        nrow (int, optional): Number of images displayed in each row of the grid.
            The Final grid size is (B / nrow, nrow). Default is 8.
        padding (int, optional): amount of padding. Default is 2.
        normalize (bool, optional): If True, shift the image to the range (0, 1),
            by subtracting the minimum and dividing by the maximum pixel value.
        range (tuple, optional): tuple (min, max) where min and max are numbers,
            then these numbers are used to normalize the image. By default, min and max
            are computed from the tensor.
        scale_each (bool, optional): If True, scale each image in the batch of
            images separately rather than the (min, max) over all images.
        pad_value (float, optional): Value for the padded pixels.
    Example:
        See this notebook `here <https://gist.github.com/anonymous/bf16430f7750c023141c562f3e9f2a91>`_
    """
    if not (torch.is_tensor(tensor) or
            (isinstance(tensor, list) and all(torch.is_tensor(t) for t in tensor))):
        raise TypeError('tensor or list of tensors expected, got {}'.format(type(tensor)))

    # if list of tensors, convert to a 4D mini-batch Tensor
    if isinstance(tensor, list):
        tensor = torch.stack(tensor, dim=0)

    if tensor.dim() == 2:  # single image H x W
        tensor = tensor.view(1, tensor.size(0), tensor.size(1))
    if tensor.dim() == 3:  # single image
        if tensor.size(0) == 1:  # if single-channel, convert to 3-channel
            tensor = torch.cat((tensor, tensor, tensor), 0)
        tensor = tensor.view(1, tensor.size(0), tensor.size(1), tensor.size(2))

    if tensor.dim() == 4 and tensor.size(1) == 1:  # single-channel images
        tensor = torch.cat((tensor, tensor, tensor), 1)

    if normalize is True:
        tensor = tensor.clone()  # avoid modifying tensor in-place
        if range is not None:
            assert isinstance(range, tuple), \
                "range has to be a tuple (min, max) if specified. min and max are numbers"

        def norm_ip(img, min, max):
            img.clamp_(min=min, max=max)
            img.add_(-min).div_(max - min + 1e-5)

        def norm_range(t, range):
            if range is not None:
                norm_ip(t, range[0], range[1])
            else:
                norm_ip(t, float(t.min()), float(t.max()))

        if scale_each is True:
            for t in tensor:  # loop over mini-batch dimension
                norm_range(t, range)
        else:
            norm_range(tensor, range)

    if tensor.size(0) == 1:
        return tensor.squeeze()

    # make the mini-batch of images into a grid
    nmaps = tensor.size(0)
    xmaps = min(nrow, nmaps)
    ymaps = int(math.ceil(float(nmaps) / xmaps))
    height, width = int(tensor.size(2) + padding), int(tensor.size(3) + padding)
    grid = tensor.new(3, height * ymaps + padding, width * xmaps + padding).fill_(pad_value)
    k = 0
    for y in irange(ymaps):
        for x in irange(xmaps):
            if k >= nmaps:
                break
            grid.narrow(1, y * height + padding, height - padding)\
                .narrow(2, x * width + padding, width - padding)\
                .copy_(tensor[k])
            k = k + 1
    return grid


def save_image(tensor, filename, nrow=8, padding=2,
               normalize=False, range=None, scale_each=False, pad_value=0):
    """Save a given Tensor into an image file.
    Args:
        tensor (Tensor or list): Image to be saved. If given a mini-batch tensor,
            saves the tensor as a grid of images by calling ``make_grid``.
        **kwargs: Other arguments are documented in ``make_grid``.
    """
    from PIL import Image
    grid = make_grid(tensor, nrow=nrow, padding=padding, pad_value=pad_value,
                     normalize=normalize, range=range, scale_each=scale_each)
    ndarr = grid.mul(255).clamp(0, 255).byte().permute(1, 2, 0).cpu().numpy()
    im = Image.fromarray(ndarr)
    im.save(filename)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# !pip install pillow --upgrade
# %reload_ext autoreload
# %autoreload

# import cv2
# from glob import glob

# input_path = '/content/gdrive/MyDrive/cs259 project/unused/jpeg_data/train_data/png_data/8_bit/*.png'

# cwd = os.getcwd()
# input_dir = os.path.join(cwd, input_path)    
# ppms = glob(input_dir)   

# img_buf = []

# # read png images to img_buf
# for count, ppm in enumerate(ppms):
#   img = Image.Image.load(ppm)
#   img_buf.append(img)
#   #img_buf.append(cv2.imread(ppm, 1))

In [ ]:
# from google.colab.patches import cv2_imshow
# cv2

In [ ]:
!pip install pillow --upgrade
%reload_ext autoreload
%autoreload

img_transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor()#,
    
#     transforms.Normalize((0.5, 0.5, 0.5), (0.24703223,  0.24348513 , 0.26158784))
])

data_path = '/content/gdrive/MyDrive/cs259 project/unused/jpeg_data/'

trainset = torchvision.datasets.ImageFolder( root=data_path+'train_data/', transform = img_transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True, num_workers=2)

testset = torchvision.datasets.ImageFolder( root=data_path+'test_data/', transform=img_transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=8,shuffle=False, num_workers=2)

Requirement already up-to-date: pillow in /usr/local/lib/python3.7/dist-packages (8.2.0)


In [ ]:
# img_transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.24703223,  0.24348513 , 0.26158784))
# ])

# trainset = datasets.ImageNet(root='/content/gdrive/MyDrive/cs259 project/imagenet', transform=img_transform)

# train_loader = torch.utils.data.DataLoader(trainset, batch_size=16,
#                                           shuffle=True, num_workers=2)

# testset = datasets.LSUN(root='/content/gdrive/MyDrive/cs259 project/imagenet', train=False,transform=img_transform)

# test_loader = torch.utils.data.DataLoader(testset, batch_size=16,shuffle=False, num_workers=2)

In [ ]:
# code block to text image loader
print(len(test_loader))
print(train_loader)
print(len(trainset))

1
46


### Parameters

In [ ]:
CHANNELS = 3
HEIGHT = 256
WIDTH = 256
EPOCHS = 200
LOG_INTERVAL = 500

### Define the network

In [ ]:
class Interpolate(nn.Module):
    def __init__(self, size, mode):
        super(Interpolate, self).__init__()
        self.interp = nn.functional.interpolate
        self.size = size
        self.mode = mode
        
    def forward(self, x):
        x = self.interp(x, size=self.size, mode=self.mode, align_corners=False)
        return x

In [ ]:
class End_to_end(nn.Module):
  def __init__(self):
    super(End_to_end, self).__init__()
    
    # Encoder
    # TODO : try with padding = 0
    
    self.conv1 = nn.Conv2d(CHANNELS, 64, kernel_size=3, stride=1, padding=1)
    self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=0)
    self.bn1 = nn.BatchNorm2d(64, affine=False)
    self.conv3 = nn.Conv2d(64, CHANNELS, kernel_size=3, stride=1, padding=1)
    
    # Decoder
    #TODO : try ConvTranspose2d
    self.interpolate = Interpolate(size=HEIGHT, mode='bilinear')
    self.deconv1 = nn.Conv2d(CHANNELS, 64, 3, stride=1, padding=1)
    self.deconv2 = nn.Conv2d(64, 64, 3, stride=1, padding=1)
    self.bn2 = nn.BatchNorm2d(64, affine=False)
    
    self.deconv_n = nn.Conv2d(64, 64, 3, stride=1, padding=1)
    self.bn_n = nn.BatchNorm2d(64, affine=False)

    
    self.deconv3 = nn.ConvTranspose2d(64, CHANNELS, 3, stride=1, padding=1)
    
    self.relu = nn.ReLU()
  
  def encode(self, x):
    out = self.relu(self.conv1(x))
    out = self.relu(self.conv2(out))
    out = self.bn1(out)
    return self.conv3(out)
    
  
  def reparameterize(self, mu, logvar):
    pass
  
  def decode(self, z):
    upscaled_image = self.interpolate(z)
    out = self.relu(self.deconv1(upscaled_image))
    out = self.relu(self.deconv2(out))
    out = self.bn2(out)
    for _ in range(10):
      out = self.relu(self.deconv_n(out))
      out = self.bn_n(out)
    out = self.deconv3(out)
    final = upscaled_image + out
    return final,out,upscaled_image

    
  def forward(self, x):
    com_img = self.encode(x)
    final,out,upscaled_image = self.decode(com_img)
    return final, out, upscaled_image, com_img, x

  


### Optimizer and loss function 

In [ ]:
CUDA = torch.cuda.is_available()
if CUDA:
  model = End_to_end().cuda()
else :
  model = end_to_end()
  
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
from skimage.metrics import normalized_root_mse, mean_squared_error
import numpy as np

In [ ]:
def loss_function(final_img,residual_img,upscaled_img,com_img,orig_img):
  # com_loss = np.sqrt(mean_squared_error(orig_img, final_img))
  # rec_loss = np.sqrt(mean_squared_error(residual_img, orig_img - upscaled_img))
  com_loss = nn.MSELoss(size_average=False)(orig_img, final_img)
  rec_loss = nn.MSELoss(size_average=False)(residual_img,orig_img-upscaled_img)
  
  return com_loss + rec_loss

### Train , Test functions

In [ ]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)
        optimizer.zero_grad()
        final, residual_img, upscaled_image, com_img, orig_im = model(data.cuda())
        loss = loss_function(final, residual_img, upscaled_image, com_img, orig_im)
        loss.backward()
        train_loss += loss.data
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data / len(data)))
            print(datetime.datetime.now().time())

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


In [ ]:
def test(epoch):
  
  model.eval()
  test_loss = 0
  for i, (data, _) in enumerate(test_loader):
        data = Variable(data, volatile=True)
        final, residual_img, upscaled_image, com_img, orig_im = model(data.cuda())
        test_loss += loss_function(final, residual_img, upscaled_image, com_img, orig_im).data
        if epoch == EPOCHS and i == 0:
#             save_image(final.data[0],'reconstruction_final',nrow=8)
#             save_image(com_img.data[0],'com_img',nrow=8)
            n = min(data.size(0), 6)
            print("saving the image "+str(n))
            comparison = torch.cat([data[:n],
              final[:n].cpu()])
            comparison = comparison.cpu()
#             print(comparison.data)
            save_image(com_img[:n].data,
                       '/content/gdrive/MyDrive/cs259 project/results2/compressed_' + str(epoch) +'.png', nrow=n)
            save_image(comparison.data,
                       '/content/gdrive/MyDrive/cs259 project/results2/reconstruction_' + str(epoch) +'.png', nrow=n)

  test_loss /= len(test_loader.dataset)
  print('====> Test set loss: {:.4f}'.format(test_loss))

### Run the code

In [ ]:
for epoch in range(1, EPOCHS+1):
    train(epoch)
    test(epoch)
#         sample = Variable(torch.randn(64, args.hidden_size),16)
#         sample = model.decode(sample)
#         print("saving im")
#         save_image(sample.data.view(64, 3, 32, 32),
# 'sample_' + str(epoch) + '.png')

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Train Epoch: 1 [0/46 (0%)]	Loss: 3214035.750000
06:29:49.372916
====> Epoch: 1 Average loss: 1757131.1250
====> Test set loss: 1902227.3750


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Train Epoch: 2 [0/46 (0%)]	Loss: 476586.625000
06:29:58.936052
====> Epoch: 2 Average loss: 325194.9688
====> Test set loss: 158905.4375
Train Epoch: 3 [0/46 (0%)]	Loss: 187501.281250
06:30:07.841580
====> Epoch: 3 Average loss: 167400.0781
====> Test set loss: 66268.9375
Train Epoch: 4 [0/46 (0%)]	Loss: 159165.781250
06:30:17.860395
====> Epoch: 4 Average loss: 130539.5234
====> Test set loss: 72783.9141
Train Epoch: 5 [0/46 (0%)]	Loss: 126496.812500
06:30:27.253402
====> Epoch: 5 Average loss: 106828.5703
====> Test set loss: 55331.2266
Train Epoch: 6 [0/46 (0%)]	Loss: 87195.953125
06:30:37.231818
====> Epoch: 6 Average loss: 88319.8516
====> Test set loss: 73177.5469
Train Epoch: 7 [0/46 (0%)]	Loss: 67256.250000
06:30:46.016929
====> Epoch: 7 Average loss: 78211.3516
====> Test set loss: 112566.8594
Train Epoch: 8 [0/46 (0%)]	Loss: 86585.796875
06:30:55.001681
====> Epoch: 8 Average loss: 70144.7734
====> Test set loss: 70603.5156
Train Epoch: 9 [0/46 (0%)]	Loss: 72087.062500
06:31:

Reference : https://github.com/L1aoXingyu/pytorch-beginner/tree/master/08-AutoEncoder

In [ ]:
#   model.save_state_dict()
torch.save(model.state_dict(), '/content/gdrive/MyDrive/cs259 project/model.pth')

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive') 

In [ ]:
model.load_state_dict(torch.load('/content/gdrive/MyDrive/cs259 project/model.pth'))

def save_images():
  epoch = EPOCHS
  model.eval()
  test_loss = 0
  
  for i, (data, _) in enumerate(test_loader):
        data = Variable(data, volatile=True)
        final, residual_img, upscaled_image, com_img, orig_im = model(data.cuda())
        test_loss += loss_function(final, residual_img, upscaled_image, com_img, orig_im).data
        if True :
#             save_image(final.data[0],'reconstruction_final',nrow=8)
#             save_image(com_img.data[0],'com_img',nrow=8)
            n = min(data.size(0), 6)
            print("saving the image "+str(i))
            comparison = torch.cat([data[:n],
              final[:n].cpu()])
            comparison = comparison.cpu()
#             print(comparison.data)
            save_image(com_img[i].data,
                         '/content/gdrive/MyDrive/cs259 project/results/compressed_' + str(i) +'.png', nrow=n)
            save_image(final[i].data,
                        '/content/gdrive/MyDrive/cs259 project/results/final_' + str(i) +'.png', nrow=n)
            save_image(orig_im[i].data,
                        '/content/gdrive/MyDrive/cs259 project/results/original_' + str(i) +'.png', nrow=n)


  test_loss /= len(test_loader.dataset)
  print('====> Test set loss: {:.4f}'.format(test_loss))

print(len(test_loader))
save_images()

1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


saving the image 0
====> Test set loss: 2265.7925


In [ ]:
!git clone https://github.com/ARM-software/SCALE-Sim

Cloning into 'SCALE-Sim'...
remote: Enumerating objects: 407, done.
remote: Total 407 (delta 0), reused 0 (delta 0), pack-reused 407
Receiving objects: 100% (407/407), 1.23 MiB | 33.17 MiB/s, done.
Resolving deltas: 100% (195/195), done.


In [ ]:
!python3 scale.py

python3: can't open file 'scale.py': [Errno 2] No such file or directory


In [ ]:
!mv original* '/content/gdrive/MyDrive/cs259 project/'

mv: cannot stat 'original*': No such file or directory


In [ ]:
print("HELLO")

HELLO
